In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam


model = Sequential()
model.add(InputLayer((1000, 4)))
model.add(LSTM(64))
model.add(Dense(8, 'relu'))
model.add(Dense(1, 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                17664     
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 18,193
Trainable params: 18,193
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import os
from tensorflow.keras.callbacks import EarlyStopping
datadir = 'sliding_window_dataset'
alldirs = os.listdir(f'{datadir}/')
for directory in alldirs:
    if os.path.isdir(f'{datadir}/{directory}') and not directory.startswith('.'):
        X = np.load(f'{datadir}/{directory}/{directory}_X.npy')
        y = np.load(f'{datadir}/{directory}/{directory}_y.npy')
        y = np.array([[i] for i in y])
        length = X.shape[0]
        test_data_idx_start = int(length * .8)
        X_train = X[:test_data_idx_start]
        y_train = y[:test_data_idx_start]
        X_test = X[test_data_idx_start:]
        y_test = y[test_data_idx_start:]
        print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
        cp_all = ModelCheckpoint(f'model_{directory}/', save_best_only=True)
        stop = EarlyStopping(monitor='val_loss', patience=10)
        model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
        model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=500, callbacks=[cp_all,stop],batch_size=256)